In [1]:
import os
import numpy as np
import pickle
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.isotonic import IsotonicRegression

output_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'data/'))

In [3]:
for gene_name in ["IL6","CXCL9","IL15","CXCL10"]:
    for exp_name in ["unstim","1hrLPS"]:

        if gene_name == "IL6":
            tss_idx = 265503
            x = np.linspace(22500000,23249998,749999).astype(int)

        elif gene_name == "CXCL9":
            tss_idx = 76928608
            x = np.arange(tss_idx-250000,tss_idx+250000+1).astype(int)

        elif gene_name == "CXCL10":
            tss_idx = 76945689
            x = np.arange(tss_idx-250000,tss_idx+250000+1).astype(int)

        elif gene_name == "IL15":
            tss_idx = 142557977
            x = np.arange(tss_idx-250000,tss_idx+250000+1).astype(int)



        if gene_name=="IL6":
            tab_1 = pd.read_table(output_path+gene_name+"/"+gene_name+"1"+exp_name+".bedgraph",header=None,names=["chr","start","end","val"])
            tab_2 = pd.read_table(output_path+gene_name+"/"+gene_name+"2"+exp_name+".bedgraph",header=None,names=["chr","start","end","val"])
        else:
            if gene_name=="IL15" or gene_name=="CXCL10":
                tab_1 = pd.read_table(output_path+gene_name+"/"+gene_name+"_"+exp_name+"_WIN21_2.bed",header=None,names=["chr","start","end","val"])
                tab_2 = pd.read_table(output_path+gene_name+"/"+gene_name+"_"+exp_name+"_WIN21_2.bed",header=None,names=["chr","start","end","val"])
            else:
                tab_1 = pd.read_table(output_path+gene_name+"/"+gene_name+"_"+exp_name+"_WIN21_1.bed",header=None,names=["chr","start","end","val"])
                tab_2 = pd.read_table(output_path+gene_name+"/"+gene_name+"_"+exp_name+"_WIN21_2.bed",header=None,names=["chr","start","end","val"])


        start_1 = np.array(tab_1[tab_1.chr == "chr4"].start).astype(int)
        end_1 = np.array(tab_1[tab_1.chr == "chr4"].end).astype(int)
        val_1 = np.array(tab_1[tab_1.chr == "chr4"].val).astype(int)

        x_vec_1 = []
        y_vec_1 = []

        for i in range(len(start_1)):
            x_vec_1.append(start_1[i])
            x_vec_1.append(end_1[i])
            y_vec_1.append(val_1[i])
            y_vec_1.append(val_1[i])

        x_vec_1 = np.array(x_vec_1)
        y_vec_1 = np.array(y_vec_1)

        y_1 = np.interp(x,x_vec_1,y_vec_1)


        start_2 = np.array(tab_2[tab_2.chr == "chr4"].start).astype(int)
        end_2 = np.array(tab_2[tab_2.chr == "chr4"].end).astype(int)
        val_2 = np.array(tab_2[tab_2.chr == "chr4"].val).astype(int)

        x_vec_2 = []
        y_vec_2 = []

        for i in range(len(start_2)):
            x_vec_2.append(start_2[i])
            x_vec_2.append(end_2[i])
            y_vec_2.append(val_2[i])
            y_vec_2.append(val_2[i])

        x_vec_2 = np.array(x_vec_2)
        y_vec_2 = np.array(y_vec_2)

        y_2 = np.interp(x,x_vec_2,y_vec_2)


        y_avg = 0.5*y_1+0.5*y_2

        background_left = IsotonicRegression(increasing=True).fit(x[0:250000],y_avg[0:250000])
        background_right = IsotonicRegression(increasing=False).fit(x[250000:],y_avg[250000:])
        y_back_left = background_left.predict(x[0:250000])
        y_back_right = background_right.predict(x[250000:])
        y_back = np.concatenate((y_back_left,y_back_right))

        with open(output_path+gene_name+'/'+exp_name+'_4C.pkl', 'wb') as file:
            pickle.dump([x,y_1,y_2,y_avg], file)